<a href="https://colab.research.google.com/github/bozhikov/SHE/blob/main/%D0%92%D0%B5%D1%80%D0%B8%D0%B3%D0%B0_%D0%BD%D0%B0_%D1%80%D0%B0%D0%B7%D0%BF%D0%B0%D0%B4_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 #%matplotlib inline
#import scipy as sp
#import matplotlib as mpl
#import matplotlib.cm as cm
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
NUM_TRAINING_RECORDS = 1000000
FIRST_CHAIN_PORTION = int(50 * NUM_TRAINING_RECORDS / 100)
SECOND_CHAIN_PORTION = int(35 * NUM_TRAINING_RECORDS / 100)
THIRD_CHAIN_PORTION = int(15 * NUM_TRAINING_RECORDS / 100)
TYPE1 = 'Type 1'
TYPE2 = 'Type 2'
TYPE3 = 'Type 3'
ENERGY_BIAS = 12000 #eV
CHANNEL_RESOLUTION = 6000
FACTOR = 10**6
# F - Type 1
# S - Type 2
# T - Type 3
TAU_F = 75.02014212 * FACTOR # μs - microseconds
TAU_S = 75.02014212 * FACTOR # μs - microseconds
TAU_T = 231.11974555 * FACTOR # μs - microseconds
E1_F = 5.6535 * FACTOR # eV
E2_F = 5.036 * FACTOR # eV
E1_S = 5.372 * FACTOR # eV
E2_S = 5.036 * FACTOR # eV
E1_T = 5.09415 * FACTOR # eV
E2_T = 4.8435 * FACTOR # eV

In [3]:
class Chain:
  def __init__(self, Type, portion, e1, e2,  tau):
    self.TYPE = Type;
    self.PORTION = portion
    self.E1 = e1
    self.E2 = e2
    self.TAU = tau
  def generate_data(self):
    self.e1_gaus = np.random.normal(self.E1, 1, self.PORTION)
    self.e2_gaus  = np.random.normal(self.E2, 1, self.PORTION)
    self.tau_pois = np.random.poisson(self.TAU, self.PORTION)
    return self.e1_gaus, self.e2_gaus, self.tau_pois


In [5]:
chains = [Chain(TYPE1, FIRST_CHAIN_PORTION, E1_F, E2_F, TAU_F), Chain(TYPE2, SECOND_CHAIN_PORTION, E1_S, E2_S, TAU_S), Chain(TYPE3, THIRD_CHAIN_PORTION, E1_T, E2_T, TAU_T)]
data = {
  'Type': [],
  'E1': [],
  'E2': [],
  'TAU': []
}

for chain in chains:
  tmp = chain.generate_data()
  # print(tmp)
  data['Type'] = np.append(data['Type'], [chain.TYPE] * len(tmp[0]))
  data['E1'] = np.append(data['E1'], tmp[0])
  data['E2'] = np.append(data['E2'], tmp[1])
  data['TAU'] = np.append(data['TAU'], tmp[2])

for col in data:
  data[col] = data[col].flatten()

data = pd.DataFrame(data)
data.head()

SyntaxError: ignored

In [ ]:
train, test = train_test_split(data, test_size=0.2)
print(len(train), len(test))
X_train = train.iloc[:, 1:4].values
y_train = train.iloc[:, 0:1].values.ravel()
X_test = test.iloc[:, 1:4].values
y_test = test.iloc[:, 0:1].values.ravel()

800000 200000


In [ ]:
model = svm.SVC(kernel='rbf', C=1E10, gamma=10, class_weight='balanced', probability=True, cache_size=7000) #try rbf (with scaling for faster computation, otherwise it would take forever)?
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#model = LogisticRegression()

In [ ]:
print(y_pred)
print(model.score(X_test, y_test))

['Type 1' 'Type 2' 'Type 3' ... 'Type 1' 'Type 1' 'Type 2']
1.0


In [ ]:
# Model Accuracy: how often is the classifier correct?
print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))
# Model Precision: what percentage of positive tuples are labeled as such?
print('Precision: ', metrics.precision_score(y_test, y_pred, average='weighted'))
# Model Recall: what percentage of positive tuples are labelled as such?
print('Recall: ', metrics.recall_score(y_test, y_pred, average='weighted'))

Accuracy:  1.0
Precision:  1.0
Recall:  1.0


In [ ]:
print(metrics.confusion_matrix(y_test,y_pred))

[[100000      0      0]
 [     0  70025      0]
 [     0      0  29975]]


In [ ]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

      Type 1       1.00      1.00      1.00    100000
      Type 2       1.00      1.00      1.00     70025
      Type 3       1.00      1.00      1.00     29975

    accuracy                           1.00    200000
   macro avg       1.00      1.00      1.00    200000
weighted avg       1.00      1.00      1.00    200000



In [ ]:
def get_type(value):
  for key in range_type.keys():
    if value >= key[0] and value <= key[1]:
      return range_type[key]
  return None

def in_range(value, range):
  return value >= range[0] and value <= range[1]

def valid_value(value):
  for r in range_by_type.values():
    e1_range = r[0]
    e2_range = r[1]
    if (value >= e1_range[0] and value <= e1_range[1]) \
        or (value >= e2_range[0] and value <= e2_range[1]):
        return True
  return False

def values_in_range(values):
  result = []
  for value in values:
    result.append(valid_value(value))
  return result

In [ ]:
records = pd.read_csv("https://alpha-decays.s3.us-east-2.amazonaws.com/caam2013newf_14.csv", sep="\t")
records.head()

,currid,strip,Ea-chanel,Ef-chanel,Timemicsec
0,2,1,913,86,208745.0
1,2,6,907,85,219673.0
2,2,0,888,84,231612.0
3,2,0,1106,103,241377.0
4,2,0,1170,109,242430.0


In [ ]:
for index, row in records.iterrows():
  candidates = []
  print(index)
  for index2, row2 in records.iterrows():
    e1 = row['Ea-chanel'] * CHANNEL_RESOLUTION * FACTOR # Channel to energy (eV)
    e2 = row2['Ea-chanel'] * CHANNEL_RESOLUTION * FACTOR # We can go the opposite way and work with channels instead
    t1_pois = np.random.poisson(row['Timemicsec'], 1000)
    tau = row2['Timemicsec'] - row['Timemicsec']
    if row['currid'] == row2['currid'] and row['strip'] < row2['strip'] and row['Timemicsec'] < row2['Timemicsec'] and tau in t1_pois: #there should be more checks
      print('found candidate!')
      candidates.append([e1, e2, tau]) # we could also save the indexes
  for candidate in candidates:
    # here we got to pick the best candidate
    # test here with +- 12 000 eV energy values?
    print(candidate)
    print(model.predict([candidate])[0]) # Type X
    print(sum(model.predict_proba([candidate])[0])) # 0 to 1?. With small datasets predict_proba gives incorrect probabilities. Alternative is model.decision_function to check how confident the model is for the prediction

0
found candidate!


Потенциални проблеми:
1. Отклонението на енергията -+ 12 000 eV не се взима предвид при обучение на модела 
2. При тестване на модела, точността му е 100%
3. Всички разпознати вериги са от тип '1'

